In [15]:
from sklearn.datasets import load_iris
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
import numpy as np
from DDAL.ddal import DDAL_detector
from DataFrame_batcher.batcher import BatchGenerator

iris = load_iris()

## Iris: case from original repo

In [16]:
df_x = pd.DataFrame(data= np.c_[iris['data'], iris['target']], columns= iris['feature_names'] + ['target']).sample(frac=1, random_state=42)
df_y = df_x.pop('target')

df_x_train = df_x[0:50]
df_y_train = df_y[0:50]


In [17]:
classifier = tree.DecisionTreeClassifier()
classifier.fit(df_x_train, df_y_train)

DecisionTreeClassifier()

In [18]:
ddal = DDAL_detector(size_batch=50, theta=0.005, lambida=0.95)

df_x_test_batch_1 = df_x[50:100]
df_y_test_batch_1 = df_y[50:100].to_frame()

df_x_test_batch_1.reset_index(inplace=True, drop=True)
df_y_test_batch_1.reset_index(inplace=True, drop=True)

In [19]:
for index, row in df_x_test_batch_1.iterrows():
    y_pred = classifier.predict_proba(df_x_test_batch_1.iloc[[index]])
    max_y_pred_prob = y_pred.max()
    ddal.count_selected_instances(max_y_pred_prob)

ddal.compute_current_density()

In [20]:
if ddal.detection_module():
    print('Drift Detected')
    ddal.reset()

## Sine test

In [22]:
data_sin = pd.read_csv(r'data\tabular\Sine.csv', index_col=0)
data_sin['Classification'] = data_sin['Classification'].map({'Initially Negative':0,'Initially Positive':1})
data_sin['Concept Drift'] = data_sin['Concept Drift'].map({'After Concept Drift Negative':0,'After Concept Drift Positive':1})

data_sin_tmp = data_sin.copy()
df_y = data_sin_tmp['Classification']

training_proportion = int(2*len(data_sin)/5000)
print(training_proportion)
df_x_train = data_sin_tmp[0:training_proportion].drop(columns=['Concept Drift','Classification'])
df_y_train = df_y[0:training_proportion]

data_test = data_sin_tmp.iloc[training_proportion:len(data_sin_tmp)].reset_index(drop=True)
classifier = tree.DecisionTreeClassifier()
classifier.fit(df_x_train, df_y_train)
y_out = np.ndarray([1,2])
batcher = BatchGenerator(500,data_test)
ddal = DDAL_detector(size_batch=500, theta=0.005, lambida=0.90)

for batch_test in batcher:
    batch_test = batch_test.drop(columns=['Concept Drift','Classification'])
    for index,_ in batch_test.iterrows():
        y_pred = classifier.predict_proba(batch_test.iloc[[index]])
        y_out = np.concatenate((y_out,y_pred))
        max_y_pred_prob = y_pred.max()
        ddal.count_selected_instances(max_y_pred_prob)

    ddal.compute_current_density()

    if ddal.detection_module():
        print('Drift Detected')
        ddal.reset()
        
    else:
        print('No drift detected')

4
No drift detected
No drift detected
No drift detected
No drift detected
No drift detected
No drift detected
No drift detected
No drift detected
No drift detected
No drift detected
No drift detected
No drift detected
No drift detected
No drift detected
No drift detected
No drift detected
No drift detected
No drift detected
No drift detected
No drift detected


In [30]:
y_out

array([[0.e+000, 1.e-323]])

In [23]:
y_out

array([[7.13767619e-09, 3.23565722e-03],
       [0.00000000e+00, 1.00000000e+00],
       [1.00000000e+00, 0.00000000e+00],
       ...,
       [0.00000000e+00, 1.00000000e+00],
       [0.00000000e+00, 1.00000000e+00],
       [0.00000000e+00, 1.00000000e+00]])

In [24]:
from sklearn.metrics import f1_score
preds = classifier.predict(data_test.drop(columns=['Concept Drift','Classification']))
f1_score(data_test['Classification'],preds)

0.808047852093529

## Very simple test

just add scalar to values to change distribution

In [25]:
data_test['x'] = data_test['x'] + 1001241
data_test['y'] = data_test['y'] - 19921341

In [26]:
y_out = np.ndarray([1,2])
batcher = BatchGenerator(500,data_test)
ddal = DDAL(size_batch=500, theta=0.005, lambida=0.90)

for batch_test in batcher:
    batch_test = batch_test.drop(columns=['Concept Drift','Classification'])
    for index,_ in batch_test.iterrows():
        y_pred = classifier.predict_proba(batch_test.iloc[[index]])
        y_out = np.concatenate((y_out,y_pred))
        max_y_pred_prob = y_pred.max()
        ddal.count_selected_instances(max_y_pred_prob)

    ddal.compute_current_density()

    if ddal.detection_module():
        print('Drift Detected')
        ddal.reset()
        
    else:
        print('No drift detected')

NameError: name 'DDAL' is not defined

In [ ]:
y_out

array([[0.e+000, 1.e-323],
       [0.e+000, 1.e+000],
       [0.e+000, 1.e+000],
       ...,
       [0.e+000, 1.e+000],
       [0.e+000, 1.e+000],
       [0.e+000, 1.e+000]])

In [ ]:
from sklearn.metrics import f1_score
preds = classifier.predict(data_test.drop(columns=['Concept Drift','Classification']))
f1_score(data_test['Classification'],preds)

0.7791145038167939

## KNN

same setup, but different classifier

In [ ]:
data_sin = pd.read_csv('../data/Sine.csv', index_col=0)
data_sin['Classification'] = data_sin['Classification'].map({'Initially Negative':0,'Initially Positive':1})
data_sin['Concept Drift'] = data_sin['Concept Drift'].map({'After Concept Drift Negative':0,'After Concept Drift Positive':1})

data_sin_tmp = data_sin.copy()
df_y = data_sin_tmp['Classification']

training_proportion = int(2*len(data_sin)/100)
print(training_proportion)
df_x_train = data_sin_tmp[0:training_proportion].drop(columns=['Concept Drift','Classification'])
df_y_train = df_y[0:training_proportion]

data_test = data_sin_tmp.iloc[training_proportion:len(data_sin_tmp)].reset_index(drop=True)
classifier = KNeighborsClassifier()
classifier.fit(df_x_train, df_y_train)
y_out = np.ndarray([1,2])
batcher = BatchGenerator(500,data_test)
ddal = DDAL(size_batch=500, theta=0.005, lambida=0.90)

for batch_test in batcher:
    batch_test = batch_test.drop(columns=['Concept Drift','Classification'])
    for index,_ in batch_test.iterrows():
        y_pred = classifier.predict_proba(batch_test.iloc[[index]])
        y_out = np.concatenate((y_out,y_pred))
        max_y_pred_prob = y_pred.max()
        ddal.count_selected_instances(max_y_pred_prob)

    ddal.compute_current_density()

    if ddal.detection_module():
        print('Drift Detected')
        ddal.reset()
        
    else:
        print('No drift detected')

200


No drift detected
Drift Detected
No drift detected
No drift detected
No drift detected
No drift detected
Drift Detected
No drift detected
No drift detected
No drift detected
No drift detected
Drift Detected
No drift detected
No drift detected
No drift detected
No drift detected
Drift Detected
No drift detected
No drift detected
No drift detected


In [ ]:
max_y_pred_prob

0.8

## Mnist

So far decision tree seems to be extremly confident in its predictions, even when it trains from two examples. On the other hand, not-fine tuned knn detects drift, when they are none. However, maybe training sample is too low